In [ ]:
!pip install -U nltk[twitter]

In [ ]:
#pip install -U nltk[all]

In [ ]:
!pip install langdetect

In [ ]:
import pandas as pd
import datetime
import re
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk import Tree

#df.sample()
pd.set_option("display.max_colwidth", 1500)
pd.set_option("display.max_rows", 10)

df=pd.read_csv(r'../input/<twitter_path/file_name>.csv')

#print(dir(df))
df.sample()

In [ ]:
df["Date"]=pd.to_datetime(df["Datetime"],format='%Y-%m-%d %H:%M:%S+00:00').dt.strftime('%Y-%m-%d')
df["DOY"]=pd.to_datetime(df["Datetime"],format='%Y-%m-%d %H:%M:%S+00:00').dt.dayofyear
df["Month"]=pd.to_datetime(df["Datetime"],format='%Y-%m-%d %H:%M:%S+00:00').dt.month
df["Quarter"]=pd.to_datetime(df["Datetime"],format='%Y-%m-%d %H:%M:%S+00:00').dt.quarter
df["Year"]=pd.to_datetime(df["Datetime"],format='%Y-%m-%d %H:%M:%S+00:00').dt.year
df.head(2)

In [ ]:
df.drop_duplicates("Tweet Id",keep=False, inplace=True)

In [ ]:
from langdetect import detect

In [ ]:
#detect language of the tweet and populate a new language column
df["Language"]=df["Text"].apply(detect)

In [ ]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

In [ ]:
#find  all synonyms for aspect mining and check them all later.
from nltk.corpus import wordnet

def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    return synonyms

word = 'inflation'
synonyms = get_synonyms(word)
print(f"Synonyms of '{word}': {synonyms}")


All the words found above are manually checked as a oneoff during the exercise. 
no match found of any of the synonym for inflation or interest rate
word rate was also used but didnt make any much difference.

In [ ]:
df["AspectInflation"]=df['Text'].str.lower().replace(' ', '').str.contains('inflation')

In [ ]:
df["AspectIntestRate"]=df['Text'].str.lower().replace(' ', '').str.contains('interestrate')

In [ ]:
df["Employment"]=df['Text'].str.lower().replace(' ', '').str.contains('employment')

In [ ]:
#apply aspect mining code and populate aspect as separate column in dataframe
#aspect mining code
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Remove stopwords
stop_words = set(stopwords.words('english'))

# Lemmatization
lemmatizer = WordNetLemmatizer()

In [ ]:
df.to_csv(r'/kaggle/working/tmp_combined_data.csv')

In [ ]:
#filterout 6 records from other languages which are es->Spanish , de-> German , and zh-ch -> Chinese
#language RO-> romanian has english words, so not filtering it
len(df)
languages_to_exclude=['es','de','zh-cn']
df=df[~df['Language'].isin(languages_to_exclude)]

**Named entity recognition tags/codes.**

JJ (adjective): Adjectives describe or modify nouns or pronouns, providing additional information about their qualities, characteristics, or attributes. In the context of NER, adjectives may not directly represent named entities themselves but can provide context or additional information about entities.

NN (noun): Nouns represent entities such as people, places, objects, or concepts. In NER, common nouns may or may not represent named entities, depending on context.

NNP (proper noun, singular): Proper nouns typically represent specific entities such as names of people, organizations, or locations. Proper nouns are often the primary focus of NER.

NNPS (proper noun, plural): Similar to NNP, NNPS represents plural proper nouns.

NNS (noun, plural): Plural nouns represent multiple instances of entities, objects, or concepts.

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk

nltk.download('averaged_perceptron_tagger_eng')
nltk.download('maxent_ne_chunker_tab')

In [ ]:
def filter_person_place(text, label,debug_mode=False):
    chunked = ne_chunk(pos_tag(word_tokenize(text)))
    if debug_mode:
        print("chunked is" +str(chunked)+ ' and count of chunked is '+str(len(chunked)))
    #print(chunked[0])
    #print('debug 1')
    #print(chunked[1])
    #print('debug 2')
    prev = None
    continuous_chunk = []
    current_chunk = []
    for subtree in chunked:
        if type(subtree) == Tree:
            if subtree.label() in label:
                if debug_mode:
                    print('debug1'+str(subtree))
                continue
            else:
                if debug_mode:
                    print('debug2'+str(subtree))
                continuous_chunk.append("".join([token for token, pos in subtree.leaves()]))
        else:
           if debug_mode:
            print('debug 3:'+str(subtree))
           #continuous_chunk.append(" ".join([token for (token, pos) in subtree]))
           continuous_chunk.append("".join(subtree[0]))
    return ' '.join(continuous_chunk)

In [ ]:
#download stopwords from corpus
from nltk.corpus import stopwords
stop = stopwords.words('english')
print(stop)

In [ ]:
#get list of australian postcodes and convert to a list, so it can be filtered out from the main twitter dataframe content.
#This filter is working. this might be a candidate to be added to the list of stop words.
#df_au_postcodes=pd.read_csv(r"https://github.com/schappim/australian-postcodes/blob/master/australian-postcodes-2021-04-23.csv")
df_au_postcodes=pd.read_csv(r'../input/au-postcodes/australian-postcodes-2021-04-23.csv')
au_postcodes_list=df_au_postcodes["Suburb"].str.lower().tolist()
#au_postcodes_list

In [ ]:
#text cleaning block.
#remove stop words. This will convert each line into a list of words
#filterout suburb names by checking if the word is appearing in au_post_codes_list. limited filtering only based on direct matching.
#ignore words with 2 or less characters
#ignore word is not a pure number
#remove apostrophe from words e.g.  '#rba', vs  '#rba’',
#remove http/s links from the text
#remove handles like @theblogger
#remove ampersand characters like '&amp;'
#including block to filter all hashtags e.g. #BRIKdolphin
remove_hashtags = lambda x: re.sub(r'#\S+', '', x)
remove_links = lambda x: re.sub(r'http\S+', '', x)
remove_handles = lambda x:re.sub('@[^\s]+','',x)
remove_ampersands = lambda x:re.sub(r'&amp;','',x)
remove_special_chars = lambda x: re.sub('[!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@‘#]', '', x)
df["text_modified"]=df["Text"].apply(lambda line:[remove_special_chars(remove_handles(remove_ampersands(remove_links(remove_hashtags(word.lower())))))
                                                           for word in line.split()
                                                           if word.lower() not in stop and len(word) > 2 and re.match('^[0-9]+$',word) is None])
df["text_modified"]=df["text_modified"].apply(lambda line:[word
                                                           for word in line
                                                           if re.match('^[0-9]+$',word) is None])
df["text_modified"]=df["text_modified"].apply(lambda line:[word
                                                           for word in line
                                                           if word not in au_postcodes_list])
df["text_modified"]

In [ ]:
df["text_modified"]

In [ ]:
#remove all non alphabetic characters e.g. emoji, star, thumsup etc
remove_non_alphabetic_characters = lambda x: re.sub(r'[^A-Za-z0–9]', '', x)
df["text_modified"]=df["text_modified"].apply(lambda line:[remove_non_alphabetic_characters(word)
                                                           for word in line])
df["text_modified"]

In [ ]:
import nltk
#nltk.download()
nltk.download('omw-1.4')

In [ ]:
df["text_modified_string"]=df["text_modified"].apply(lambda x : ' '.join(x))

In [ ]:
#this is done after cleaning the data now, since it was introducing spaces in the hyperlink and causing problems while filtering all hyperlinks.
#DO NOT CHANGE the position of this transformer
df["text_modified_string"]=df["text_modified_string"].apply(lambda line:filter_person_place(line,['GPE','PERSON']))
df["text_modified_string"]

In [ ]:
df.to_csv(r'/kaggle/working/tmp_cleaned_data.csv')

In [ ]:
'''import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_df=1, min_df=1, token_pattern='\w+|\$[\d\.]+|\S+')

# Get distinct year and quarter combinations
distinct_year_quarters = df[['Year', 'Quarter']].drop_duplicates()
print (distinct_year_quarters)

# Loop through each year-quarter combination
for _, row in distinct_year_quarters.iterrows():
    #print('debug 1'+str(row))
    year = row['Year']
    quarter = row['Quarter']
    
    # Filter records for the current year and quarter
    subset = df[(df['Year'] == year) & (df['Quarter'] == quarter)]
    
    # Process the subset of records
    print(f"Processing records for Year: {year}, Quarter: {quarter}")
    #print(subset["text_modified_string"])
    tf = vectorizer.fit_transform(subset["text_modified_string"].dropna()).toarray()
    tf_feature_names = vectorizer.get_feature_names_out()

    from sklearn.decomposition import LatentDirichletAllocation
    number_of_topics = 10
    model = LatentDirichletAllocation(n_components=number_of_topics, random_state=45) # random state for reproducibility
    # Fit data to model
    model.fit(tf)

    no_top_words = 20
    #compile display_topics function def  given in cell below
    topics=display_topics(model, tf_feature_names, no_top_words)
    topics

    topics.to_csv(r'/kaggle/working/generated_topics_50_year_'+str(year)+'_quarter_'+str(quarter)+'.csv')
    
    
    # Add your processing logic here
    # For example, save to a separate file or perform some operations
    # subset.to_csv(f'output_{year}_Q{quarter}.csv', index=False)
'''

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# the vectorizer object will be used to transform text to vector form
#setting max_df and min_df to default values of 1 to not ignore any values.
vectorizer = CountVectorizer(max_df=1, min_df=1, token_pattern='\w+|\$[\d\.]+|\S+')
#regex word string may contain $ or . with number and is a non empty string
#above regex need to be changed to pick all words not starting with a number or do this filtering earlier.
#vectorizer = CountVectorizer(token_pattern='^[^0-9]+\w.$')


# apply transformation
#tf = vectorizer.fit_transform(df["text_stemmed_string"]).toarray()#commenting after skipping stemming and lemmatization
tf = vectorizer.fit_transform(df["text_modified_string"]).toarray()

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names_out()

In [ ]:
#these are the predominant topics
tf_feature_names

In [ ]:
len(tf_feature_names)
#len(set(tf_feature_names))
#tf_feature_names

In [ ]:
#filter out links and special characters
#tf_feature_names
tf

In [ ]:
tf.shape

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
number_of_topics = 10
model = LatentDirichletAllocation(n_components=number_of_topics, random_state=45) # random state for reproducibility
# Fit data to model
model.fit(tf)

In [ ]:
def display_topics(model, feature_names, no_top_words):
    topic_dict = {}
    for topic_idx, topic in enumerate(model.components_):
        topic_dict["Topic %d words" % (topic_idx)]= ['{}'.format(feature_names[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
        topic_dict["Topic %d weights" % (topic_idx)]= ['{:.1f}'.format(topic[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
    return pd.DataFrame(topic_dict)

In [ ]:
no_top_words = 50
topics=display_topics(model, tf_feature_names, no_top_words)
topics

In [ ]:
topics.to_csv(r'/kaggle/working/generated_topics_50.csv')

In [ ]:
#now only pick the records where aspect mining is detected and then apply thematic analysis.
from sklearn.feature_extraction.text import CountVectorizer

# the vectorizer object will be used to transform text to vector form
#setting max_df and min_df to default values of 1 to not ignore any values.
vectorizer = CountVectorizer(max_df=1, min_df=1, token_pattern='\w+|\$[\d\.]+|\S+')
#regex word string may contain $ or . with number and is a non empty string
#above regex need to be changed to pick all words not starting with a number or do this filtering earlier.
#vectorizer = CountVectorizer(token_pattern='^[^0-9]+\w.$')


# apply transformation
#tf = vectorizer.fit_transform(df["text_stemmed_string"]).toarray()#commenting this and adding next line because we skipped stemming and lemmatization
tf = vectorizer.fit_transform(df[(df['AspectInflation'] == True) | (df['AspectIntestRate'] == True) | (df['Employment'] == True)]["text_modified_string"].dropna()).toarray()

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names_out()

In [ ]:
#these are the predominant topics
tf_feature_names

In [ ]:
len(tf_feature_names)
#len(set(tf_feature_names))
#tf_feature_names

In [ ]:
#filter out links and special characters
#tf_feature_names
tf

In [ ]:
tf.shape

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
number_of_topics = 10
model = LatentDirichletAllocation(n_components=number_of_topics, random_state=45) # random state for reproducibility
# Fit data to model
model.fit(tf)

In [ ]:
def display_topics(model, feature_names, no_top_words):
    topic_dict = {}
    for topic_idx, topic in enumerate(model.components_):
        topic_dict["Topic %d words" % (topic_idx)]= ['{}'.format(feature_names[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
        topic_dict["Topic %d weights" % (topic_idx)]= ['{:.1f}'.format(topic[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
    return pd.DataFrame(topic_dict)

In [ ]:
no_top_words = 50
topics=display_topics(model, tf_feature_names, no_top_words)
topics

In [ ]:
topics.to_csv(r'/kaggle/working/generated_topics_50_after_aspect_mining.csv')